# Problem statement

In today’s world, information is everywhere. It has a ton of advantages, but it does not mean that it comes with zero cost. On the contrary, information overload is a serious problem as consumers can feel overwhelmed and either decide not to buy anything or do not take all options into account. To remedy this problem, a big food company for which you are working as data scientist wants to develop a recommendation system (RS) to provide better service to its customers.

RS can be defined as a system that makes personalized recommendations from a large range of different options by implicitly or explicitly eliciting the user’s preference for a product. RS benefits consumers as they reduce consumers' effort and search costs by making the decision process quick, easier, and painless.

As a data scientist at a big food company, you are asked to run item. Basically, user-based recommendation is a technique used to predict the items that a user might like based on ratings given to that item by the other users who have similar taste with that of the target user.

For this task, you are provided ‘recom.csv’ dataset. In this dataset, you have following columns:

•	Main_ID (Customer ID)

•	Transaction_ID

•	Date

•	Price

•	Code_Product

•	Amount (Order Amount)

•	ItemKey

Well, using this data, you are asked to generate a dataframe in which you recommend an item (or items) for each user.

Hints: 
1)	Please note that there is no rating in the dataset, but, no worries, there is a way to solve this problem.
2)	If possible, please recommend multiple items per user.
3)	You can consider creating pivot table with ‘Main_ID’, ‘ItemKey’, ‘Transaction_ID’.


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import random
import numpy as np

In [2]:
df = pd.read_csv('recom.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      50000 non-null  int64  
 1   Main_ID         50000 non-null  object 
 2   Transaction_ID  50000 non-null  object 
 3   Date            50000 non-null  object 
 4   Price           50000 non-null  float64
 5   Code_Product    50000 non-null  float64
 6   Amount          50000 non-null  float64
 7   ItemKey         28597 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 3.1+ MB


In [4]:
df.head()

,Unnamed: 0,Main_ID,Transaction_ID,Date,Price,Code_Product,Amount,ItemKey
0,0,90fada91,264f7a69,2022-10-07 20:53:49.153,125.0,5002.0,1.0,5002.0
1,1,9006f9ac,45c7d853,2022-09-17 15:54:57.187,19.0,35012.0,1.0,NaN
2,2,32270891,61ad76dd,2022-11-28 13:51:55.667,141.0,5005.0,1.0,5005.0
3,3,97e03e47,41ee09f6,2022-09-12 16:20:22.110,4.5,35078.5,1.0,NaN
4,4,41949228,244fe6d8,2022-10-14 18:53:43.933,129.5,49291.5,5.0,NaN


In [5]:
# Dropping the unnamed column which represents the index.
df.drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
# Renaming the column Main_ID as Customer_ID.
df.rename(columns={'Main_ID': 'Customer_ID'}, inplace=True)

In [7]:
# Renaming as per the problem statement
df.rename(columns={'Amount': 'Order_Amount'}, inplace=True)

In [8]:
# Checking for duplicate rows in the dataset.
duplicate_rows = len(df[df.duplicated()])
print('Number of duplicate rows in the dataset: {}'.format(duplicate_rows))

Number of duplicate rows in the dataset: 171


In [9]:
# Dropping the duplicate entries and reset the index.
df = df.drop_duplicates().reset_index(drop=True)

In [10]:
duplicate_rows = len(df[df.duplicated()])
print('Number of duplicate rows in the dataset: {}'.format(duplicate_rows))

Number of duplicate rows in the dataset: 0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49829 entries, 0 to 49828
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Customer_ID     49829 non-null  object 
 1   Transaction_ID  49829 non-null  object 
 2   Date            49829 non-null  object 
 3   Price           49829 non-null  float64
 4   Code_Product    49829 non-null  float64
 5   Order_Amount    49829 non-null  float64
 6   ItemKey         28512 non-null  float64
dtypes: float64(4), object(3)
memory usage: 2.7+ MB


In [12]:
df.isna().sum()

Customer_ID           0
Transaction_ID        0
Date                  0
Price                 0
Code_Product          0
Order_Amount          0
ItemKey           21317
dtype: int64

There are missing values in the ItemKey column. We see that there are some identical values in Code_Product and Item_Key.

In [13]:
# Check for identical values in the columns: Code_Product and ItemKey
df[df['Code_Product']  == df['ItemKey']]

,Customer_ID,Transaction_ID,Date,Price,Code_Product,Order_Amount,ItemKey
0,90fada91,264f7a69,2022-10-07 20:53:49.153,125.0,5002.0,1.0,5002.0
2,32270891,61ad76dd,2022-11-28 13:51:55.667,141.0,5005.0,1.0,5005.0
6,4cced191,87c4b548,2022-10-15 17:00:39.867,6.0,5012.0,1.0,5012.0
9,2be09449,a2d55439,2022-10-28 12:23:00.727,90.0,5011.5,1.0,5011.5
10,45f3b09a,2d923b3a,2022-10-07 15:50:01.013,33.5,5025.0,1.0,5025.0
...,...,...,...,...,...,...,...
49822,d3754d5f,0b8292d9,2022-10-30 12:51:33.277,87.5,45004.0,2.0,45004.0
49823,c8d55170,80f65a13,2022-11-27 13:45:08.837,42.5,45001.5,1.0,45001.5
49824,bb127ffb,4e0eb5ab,2022-09-24 21:48:20.847,111.5,45004.0,4.0,45004.0
49826,c983862a,d1a35c5c,2022-11-24 20:02:43.023,178.0,5001.5,1.0,5001.5


In [14]:
# There are some missing values in ItemKey column
df[(df['Code_Product']  != df['ItemKey']) & df['ItemKey'].isnull()]

,Customer_ID,Transaction_ID,Date,Price,Code_Product,Order_Amount,ItemKey
1,9006f9ac,45c7d853,2022-09-17 15:54:57.187,19.0,35012.0,1.0,NaN
3,97e03e47,41ee09f6,2022-09-12 16:20:22.110,4.5,35078.5,1.0,NaN
4,41949228,244fe6d8,2022-10-14 18:53:43.933,129.5,49291.5,5.0,NaN
5,0e816717,654c3866,2022-10-21 20:53:45.037,72.0,49291.5,1.0,NaN
7,d93c0430,4e39a356,2022-10-21 10:50:28.717,33.5,49292.0,1.0,NaN
...,...,...,...,...,...,...,...
49818,bb3c0a55,7cc8cfe2,2022-11-29 12:53:09.393,96.0,49130.0,1.0,NaN
49819,d2879869,cb98451f,2022-10-22 20:20:11.290,12.5,35075.5,1.0,NaN
49821,00ab8ebd,6272da05,2022-11-28 15:00:31.897,8.0,35078.5,2.0,NaN
49825,a8bc484a,c9946c16,2022-11-18 19:49:01.973,34.0,49292.0,1.0,NaN


Dropping the column: 'ItemKey' as ItemKey and Code_Product are one and the same. 

In [15]:
df.drop('ItemKey', axis=1, inplace=True)

In [16]:
df.head()

,Customer_ID,Transaction_ID,Date,Price,Code_Product,Order_Amount
0,90fada91,264f7a69,2022-10-07 20:53:49.153,125.0,5002.0,1.0
1,9006f9ac,45c7d853,2022-09-17 15:54:57.187,19.0,35012.0,1.0
2,32270891,61ad76dd,2022-11-28 13:51:55.667,141.0,5005.0,1.0
3,97e03e47,41ee09f6,2022-09-12 16:20:22.110,4.5,35078.5,1.0
4,41949228,244fe6d8,2022-10-14 18:53:43.933,129.5,49291.5,5.0


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49829 entries, 0 to 49828
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Customer_ID     49829 non-null  object 
 1   Transaction_ID  49829 non-null  object 
 2   Date            49829 non-null  object 
 3   Price           49829 non-null  float64
 4   Code_Product    49829 non-null  float64
 5   Order_Amount    49829 non-null  float64
dtypes: float64(3), object(3)
memory usage: 2.3+ MB


In [18]:
df.isnull().sum()

Customer_ID       0
Transaction_ID    0
Date              0
Price             0
Code_Product      0
Order_Amount      0
dtype: int64

In [19]:
num_users = df['Customer_ID'].nunique()
print(f'Number of users in the dataset: {num_users}')
num_transactions = df['Transaction_ID'].nunique()
print(f'Number of transactions in the dataset: {num_transactions}')
num_products = df['Code_Product'].nunique()
print(f'Number of unique product codes in the dataset: {num_products}')

Number of users in the dataset: 28514
Number of transactions in the dataset: 48403
Number of unique product codes in the dataset: 333


In [20]:
df['Transaction_ID'].value_counts()

6dc95f40    3
909e7ee8    3
af383319    3
fb6b2eea    3
35e94b12    3
           ..
cdf80e4f    1
34186182    1
0226cf74    1
daed5eda    1
0ff8b41f    1
Name: Transaction_ID, Length: 48403, dtype: int64

In [21]:
df[df['Code_Product'] == 45003.0]

,Customer_ID,Transaction_ID,Date,Price,Code_Product,Order_Amount
120,47627ecc,91434c38,2022-09-02 22:08:45.323,2.5,45003.0,1.0
387,bb039218,2f3e14e6,2022-10-02 12:47:28.810,1.0,45003.0,1.0
521,7995883f,c990ddc8,2022-11-08 16:31:07.507,0.5,45003.0,1.0
950,aa39f921,bdd78870,2022-09-26 13:05:02.087,30.5,45003.0,1.0
1532,9fa49207,dd0727c6,2022-12-01 18:07:14.707,2.0,45003.0,1.0
...,...,...,...,...,...,...
48249,a59967fe,7b3ac5d8,2022-09-28 12:01:49.047,1.0,45003.0,1.0
48435,582b07f4,68a0b12b,2022-11-05 18:14:56.950,92.5,45003.0,4.0
48455,8286fffa,5c6c638c,2022-10-21 17:33:01.860,44.5,45003.0,1.0
48478,aad7e10e,e8c80de8,2022-10-28 18:48:35.920,90.0,45003.0,1.0


In [22]:
df[df['Transaction_ID'] == '35e94b12']

,Customer_ID,Transaction_ID,Date,Price,Code_Product,Order_Amount
17005,e5d42351,35e94b12,2022-11-01 14:22:50.560,212.0,45003.0,1.0
23992,e5d42351,35e94b12,2022-11-01 14:22:50.560,212.0,45004.0,1.0
48413,e5d42351,35e94b12,2022-11-01 14:22:50.560,212.0,15008.5,3.0


In [23]:
df[df['Transaction_ID'] == 'c9bbbbcf']

,Customer_ID,Transaction_ID,Date,Price,Code_Product,Order_Amount
11008,7b12c606,c9bbbbcf,2022-10-14 17:33:00.467,122.0,30021.5,1.0
11033,7b12c606,c9bbbbcf,2022-10-14 17:33:00.467,122.0,10013.0,1.0
16985,7b12c606,c9bbbbcf,2022-10-14 17:33:00.467,122.0,30009.0,1.0


In [24]:
df[df['Code_Product'] == 30009.0]

,Customer_ID,Transaction_ID,Date,Price,Code_Product,Order_Amount
190,7bced1c0,84c66643,2022-11-28 18:55:58.587,6.5,30009.0,1.0
465,a4277e34,1b513e59,2022-10-20 15:40:44.837,6.5,30009.0,1.0
485,6a0fa6e8,346c3a1e,2022-10-30 12:31:59.027,10.5,30009.0,1.0
851,ec714667,a6c57e2b,2022-11-12 21:08:47.943,7.5,30009.0,1.0
1063,81d36e76,6c695f3e,2022-10-28 11:00:18.200,18.0,30009.0,1.0
...,...,...,...,...,...,...
48106,d2bb40b4,ca40fc63,2022-11-15 20:54:02.453,46.0,30009.0,1.0
48563,08d4ab12,609bccab,2022-11-10 16:56:48.157,73.0,30009.0,1.0
49079,c613eb13,1d4e855c,2022-10-15 13:49:34.877,79.5,30009.0,1.0
49522,cbec6813,fa05b186,2022-10-08 15:51:47.870,11.0,30009.0,1.0


In [25]:
df[df['Transaction_ID'] == '1b513e59']

,Customer_ID,Transaction_ID,Date,Price,Code_Product,Order_Amount
465,a4277e34,1b513e59,2022-10-20 15:40:44.837,6.5,30009.0,1.0


In [26]:
df[df['Transaction_ID'] == '061264fd']

,Customer_ID,Transaction_ID,Date,Price,Code_Product,Order_Amount
49702,c34e9b3b,061264fd,2022-10-03 18:08:28.063,24.0,30009.0,2.0


In [27]:
df[df['Transaction_ID'] == '84c66643']

,Customer_ID,Transaction_ID,Date,Price,Code_Product,Order_Amount
190,7bced1c0,84c66643,2022-11-28 18:55:58.587,6.5,30009.0,1.0


In [28]:
df[df['Transaction_ID'] == 'fa05b186']

,Customer_ID,Transaction_ID,Date,Price,Code_Product,Order_Amount
49522,cbec6813,fa05b186,2022-10-08 15:51:47.870,11.0,30009.0,1.0


In [29]:
df['Order_Amount'].value_counts()

1.0     41528
2.0      6365
3.0      1279
4.0       415
5.0       130
6.0        56
7.0        16
10.0       11
8.0         7
20.0        4
13.0        4
18.0        3
9.0         2
54.0        1
33.0        1
19.0        1
41.0        1
17.0        1
28.0        1
14.0        1
16.0        1
12.0        1
Name: Order_Amount, dtype: int64

In this dataset, there is no explicit rating provided by the user for the given products.

In [30]:
df.rename(columns={'Date': 'Timestamp'}, inplace=True)

In [31]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49829 entries, 0 to 49828
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Customer_ID     49829 non-null  object        
 1   Transaction_ID  49829 non-null  object        
 2   Timestamp       49829 non-null  datetime64[ns]
 3   Price           49829 non-null  float64       
 4   Code_Product    49829 non-null  float64       
 5   Order_Amount    49829 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 2.3+ MB


In [33]:
df.head()

,Customer_ID,Transaction_ID,Timestamp,Price,Code_Product,Order_Amount
0,90fada91,264f7a69,2022-10-07 20:53:49.153,125.0,5002.0,1.0
1,9006f9ac,45c7d853,2022-09-17 15:54:57.187,19.0,35012.0,1.0
2,32270891,61ad76dd,2022-11-28 13:51:55.667,141.0,5005.0,1.0
3,97e03e47,41ee09f6,2022-09-12 16:20:22.110,4.5,35078.5,1.0
4,41949228,244fe6d8,2022-10-14 18:53:43.933,129.5,49291.5,5.0


### Train Test split

In [34]:
# Perform train and test split
train, test = train_test_split(df, 
                               test_size=0.05, 
                               random_state=100)

In [35]:
num_train_users = train['Customer_ID'].nunique()
print(f'Number of users in the train dataset: {num_train_users}')
num_train_transactions = train['Transaction_ID'].nunique()
print(f'Number of transactions in the train dataset: {num_train_transactions}')
num_train_products = train['Code_Product'].nunique()
print(f'Number of unique product codes in the train dataset: {num_train_products}')

Number of users in the train dataset: 27472
Number of transactions in the train dataset: 46050
Number of unique product codes in the train dataset: 330


In [36]:
num_test_users = test['Customer_ID'].nunique()
print(f'Number of users in the test dataset: {num_test_users}')
num_test_transactions = test['Transaction_ID'].nunique()
print(f'Number of transactions in the test dataset: {num_test_transactions}')
num_test_products = test['Code_Product'].nunique()
print(f'Number of unique product codes in the test dataset: {num_test_products}')

Number of users in the test dataset: 2278
Number of transactions in the test dataset: 2490
Number of unique product codes in the test dataset: 172


### Product recommendation for new users

In [37]:
product_df = train['Code_Product'].value_counts().reset_index()
product_df.columns = {'Purchase_Frequency', 'Code_Product'}
product_df.head()

,Code_Product,Purchase_Frequency
0,49292.0,4589
1,49291.5,4403
2,5000.5,3603
3,45004.0,2924
4,10013.0,1484


In [38]:
date_min = train['Timestamp'].min()
date_max = train['Timestamp'].max()

print(f'Min date of purchase history in the train dataset: {date_min}')
print(f'Max date of purchase history in the train dataset: {date_max}')

Min date of purchase history in the train dataset: 2022-08-26 00:00:14.643000
Max date of purchase history in the train dataset: 2022-12-03 02:14:57.983000


In [39]:
purchase_recency = train.groupby('Code_Product')['Timestamp'].max().reset_index()
purchase_recency['Purchase_Recency_Days'] = (date_max - purchase_recency['Timestamp']).dt.days
purchase_recency.head()

,Code_Product,Timestamp,Purchase_Recency_Days
0,5000.5,2022-12-02 22:21:30.640,0
1,5001.0,2022-12-02 21:24:57.480,0
2,5001.5,2022-12-02 22:45:33.607,0
3,5002.0,2022-12-02 18:51:35.360,0
4,5002.5,2022-11-24 19:09:04.790,8


In [40]:
purchase_value = train.groupby('Code_Product')['Order_Amount'].sum().reset_index()
purchase_value.rename(columns={'Order_Amount': 'Purchase_Value'}, inplace=True)
purchase_value.head()

,Code_Product,Purchase_Value
0,5000.5,4484.0
1,5001.0,623.0
2,5001.5,487.0
3,5002.0,761.0
4,5002.5,5.0


In [41]:
product_df = product_df.merge(purchase_recency, on="Code_Product", how='left')
product_df.drop(columns='Timestamp', inplace=True)
product_df = product_df.merge(purchase_value, on="Code_Product", how='left')

In [42]:
product_df.head()

,Code_Product,Purchase_Frequency,Purchase_Recency_Days,Purchase_Value
0,49292.0,4589,0,5675.0
1,49291.5,4403,0,5359.0
2,5000.5,3603,0,4484.0
3,45004.0,2924,0,4304.0
4,10013.0,1484,0,1614.0


In [43]:
popular_products = product_df.sort_values(['Purchase_Value',
                                           'Purchase_Recency_Days',
                                           'Purchase_Frequency', 
                                          ],
                                          ascending=[False, True, False])
popular_products = popular_products.set_index('Code_Product')

In [44]:
popular_products.head(10)

,Purchase_Frequency,Purchase_Recency_Days,Purchase_Value
Code_Product,,,
49292.0,4589,0,5675.0
49291.5,4403,0,5359.0
5000.5,3603,0,4484.0
45004.0,2924,0,4304.0
10013.0,1484,0,1614.0
5009.0,1185,0,1371.0
48504.5,992,0,1225.0
25003.0,1009,0,1182.0
45001.0,888,0,1158.0


In [45]:
def get_popular_products(top_n=3):
    return popular_products.index.to_list()[:top_n]

### Item based Collaborative Filtering

In [46]:
# Create item based matrix.
item_based_matrix = train.pivot_table(index='Customer_ID', 
                                      columns='Code_Product',
                                      values='Transaction_ID',
                                      aggfunc='count').T

In [47]:
item_based_matrix.head()

Customer_ID,00024de6,00084856,0008e848,00096930,000c66b7,000e98ee,00165e2e,00171ffd,0019e439,001a93a4,...,ffe473f3,ffe9b45e,ffea59de,ffec4938,fff4483d,fff8b1c4,fff905d0,fff9726b,fffa332b,ffff4aa9
Code_Product,,,,,,,,,,,,,,,,,,,,,
5000.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5001.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5002.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5002.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
num_rows = item_based_matrix.shape[0]
num_cols = item_based_matrix.shape[1]
print(f'Total num of elements : {num_rows*num_cols}')

Total num of elements : 9065760


In [49]:
count = (item_based_matrix >= 1).sum().sum()
print(f'Number of elements >= 1 : [{count}]')

Number of elements >= 1 : [43775]


In [50]:
item_based_matrix.shape

(330, 27472)

In [51]:
item_based_matrix.min().min()

1.0

In [52]:
item_based_matrix.max().max()

10.0

In [53]:
dummy_item_based_matrix = item_based_matrix.copy()
dummy_item_based_matrix = dummy_item_based_matrix.fillna(0)

# Since we do not want to recommend the products which are already bought by the customer
dummy_item_based_matrix = dummy_item_based_matrix.applymap(lambda x: 0 if x >= 1 else 1)
dummy_item_based_matrix.head()

Customer_ID,00024de6,00084856,0008e848,00096930,000c66b7,000e98ee,00165e2e,00171ffd,0019e439,001a93a4,...,ffe473f3,ffe9b45e,ffea59de,ffec4938,fff4483d,fff8b1c4,fff905d0,fff9726b,fffa332b,ffff4aa9
Code_Product,,,,,,,,,,,,,,,,,,,,,
5000.5,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5001.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5001.5,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5002.0,1,1,1,1,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5002.5,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [54]:
count = (dummy_item_based_matrix >= 1).sum().sum()
print(f'Number of elements >= 1 : [{count}]')

Number of elements >= 1 : [9021985]


In [55]:
dummy_item_based_matrix.shape

(330, 27472)

In [56]:
dummy_item_based_matrix.min().min()

0

In [57]:
dummy_item_based_matrix.max().max()

1

We need to consider user biases to account for the variations in scales among different users in item based collaborative filtering.

In [58]:
# find the mean value of each user
mean = np.nanmean(item_based_matrix, axis=1)
normalized_item_based_matrix = (item_based_matrix.T - mean).T

In [59]:
normalized_item_based_matrix.shape

(330, 27472)

In [60]:
normalized_item_based_matrix.head()

Customer_ID,00024de6,00084856,0008e848,00096930,000c66b7,000e98ee,00165e2e,00171ffd,0019e439,001a93a4,...,ffe473f3,ffe9b45e,ffea59de,ffec4938,fff4483d,fff8b1c4,fff905d0,fff9726b,fffa332b,ffff4aa9
Code_Product,,,,,,,,,,,,,,,,,,,,,
5000.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5001.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5002.0,NaN,NaN,NaN,NaN,-0.06951,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5002.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
normalized_item_based_matrix.max().max()

7.590909090909091

In [62]:
normalized_item_based_matrix.min().min()

-1.4090909090909092

In [63]:
normalized_item_based_matrix = normalized_item_based_matrix.fillna(0)

In [64]:
# Find the similarity between the items using the item based matrix
item_similarity = cosine_similarity(normalized_item_based_matrix)

In [65]:
item_similarity.shape

(330, 330)

In [66]:
item_similarity_df = pd.DataFrame(item_similarity, 
                                  index = normalized_item_based_matrix.index,
                                  columns = normalized_item_based_matrix.index)

In [67]:
item_similarity_df.head()

Code_Product,5000.5,5001.0,5001.5,5002.0,5002.5,5003.0,5004.0,5004.5,5005.0,5007.0,...,200035.5,200037.0,200037.5,200038.0,200038.5,200045.5,200046.0,200046.5,200047.0,350027.5
Code_Product,,,,,,,,,,,,,,,,,,,,,
5000.5,1.000000,0.106052,0.075422,0.112099,0.0,0.0,0.0,0.024155,0.035644,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5001.0,0.106052,1.000000,-0.008833,0.088176,0.0,0.0,0.0,-0.011239,0.214897,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5001.5,0.075422,-0.008833,1.000000,-0.009468,0.0,0.0,0.0,-0.003781,-0.015631,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5002.0,0.112099,0.088176,-0.009468,1.000000,0.0,0.0,0.0,0.189724,-0.020558,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5002.5,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
print(f'Min value in the item similarity matrix : {item_similarity_df.min().min()}')
print(f'Max value in the item similarity matrix: {item_similarity_df.max().max()}')

Min value in the item similarity matrix : -0.2866204811879405
Max value in the item similarity matrix: 1.000000000000115


In [69]:
# In this case, we are interested only in postively correlated items.
item_similarity_df[item_similarity_df < 0] = 0
item_similarity_df.head()

Code_Product,5000.5,5001.0,5001.5,5002.0,5002.5,5003.0,5004.0,5004.5,5005.0,5007.0,...,200035.5,200037.0,200037.5,200038.0,200038.5,200045.5,200046.0,200046.5,200047.0,350027.5
Code_Product,,,,,,,,,,,,,,,,,,,,,
5000.5,1.000000,0.106052,0.075422,0.112099,0.0,0.0,0.0,0.024155,0.035644,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5001.0,0.106052,1.000000,0.000000,0.088176,0.0,0.0,0.0,0.000000,0.214897,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5001.5,0.075422,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5002.0,0.112099,0.088176,0.000000,1.000000,0.0,0.0,0.0,0.189724,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5002.5,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
print(f'Min value in the item similarity matrix : {item_similarity_df.min().min()}')
print(f'Min value in the item similarity matrix : {item_similarity_df.max().max()}')

Min value in the item similarity matrix : 0.0
Min value in the item similarity matrix : 1.000000000000115


In [71]:
item_based_matrix.T.shape

(27472, 330)

In [72]:
item_similarity_df.shape

(330, 330)

In [73]:
dummy_item_based_matrix.shape

(330, 27472)

In [74]:
# We are using dummy_item_based_matrix to ensure that we are not recommending the products
# which are already bought by the customer.
item_scores = np.dot(dummy_item_based_matrix.T.fillna(0), item_similarity_df)

In [75]:
item_scores.shape

(27472, 330)

In [76]:
item_scores_df = pd.DataFrame(item_scores, 
                              index = dummy_item_based_matrix.T.index.values,
                              columns = item_similarity_df.index.values
                             )
item_scores_df.head()

,5000.5,5001.0,5001.5,5002.0,5002.5,5003.0,5004.0,5004.5,5005.0,5007.0,...,200035.5,200037.0,200037.5,200038.0,200038.5,200045.5,200046.0,200046.5,200047.0,350027.5
00024de6,4.279142,3.583868,1.537089,2.344417,0.0,0.0,0.0,1.392058,3.680987,0.0,...,1.439315,0.0,1.029745,0.0,0.0,1.360513,0.0,0.0,0.0,0.0
00084856,3.978187,3.450365,1.395837,2.348631,0.0,0.0,0.0,1.339659,3.353217,0.0,...,1.439315,0.0,1.029745,0.0,0.0,1.360513,0.0,0.0,0.0,0.0
0008e848,4.253358,3.583687,1.537741,2.368783,0.0,0.0,0.0,1.392058,3.680987,0.0,...,1.439315,0.0,1.029745,0.0,0.0,1.360513,0.0,0.0,0.0,0.0
00096930,4.133114,3.520846,1.447266,2.358607,0.0,0.0,0.0,1.381669,3.502704,0.0,...,1.439315,0.0,1.029745,0.0,0.0,1.360513,0.0,0.0,0.0,0.0
000c66b7,4.121801,3.458786,1.525168,1.350042,0.0,0.0,0.0,1.201735,3.680987,0.0,...,1.439315,0.0,1.029745,0.0,0.0,1.360513,0.0,0.0,0.0,0.0


In [77]:
item_scores_df.min().min()

0.0

In [78]:
item_scores_df.max().max()

4.304279154350646

In [79]:
new_customers = []
old_customers = []

In [80]:
def generate_product_recommendations(cust_id, top_n=3):
    
    # Product recommendations for the existing users in the platform
    if cust_id in item_based_matrix.T.index:
        old_customers.append(cust_id)
        # Access the row by customer id
        cust_row = item_scores_df.loc[cust_id]
        # Sort values in descending order
        cust_row = cust_row.sort_values(ascending=False)
        # Get the top n recommended items
        recommended_items = cust_row.index.to_list()[:top_n]
    else: # Product recommendations for the new users
        new_customers.append(cust_id)
        recommended_items = get_popular_products(top_n)

    return recommended_items

In [81]:
# Generate recommendations for each user in the test set
cust_item_recommendations = {}
top_n = 3

for customer in test['Customer_ID'].unique():
    recommended_items = generate_product_recommendations(customer, top_n=top_n)
    cust_item_recommendations[customer] = recommended_items

In [82]:
len(new_customers)

1042

In [83]:
# Create recommendation dataframe
col_names = ['Top'+ '_' + str(i) for i in range(1, top_n+1)]
recommendation_df = pd.DataFrame.from_dict(cust_item_recommendations, 
                                           orient='index', 
                                           columns=col_names)


In [84]:
recommendation_df.head()

,Top_1,Top_2,Top_3
813a4cf2,5000.5,10031.0,5005.0
af5f64b4,49292.0,49291.5,5000.5
b3ce7f25,10031.0,5005.0,10006.0
f84a035b,5000.5,10031.0,5005.0
7270dcae,49292.0,49291.5,5000.5


### Evaluation of test set

In [85]:
actual_test = test.groupby('Customer_ID')['Code_Product'].unique()
len(actual_test)

2278

In [86]:
train_cust_bought_product = train.groupby('Customer_ID')['Code_Product'].unique().to_dict()
len(train_cust_bought_product)

27472

In [87]:
filtered_dict = {key: value for key, value in train_cust_bought_product.items() if key in actual_test.index}
len(filtered_dict)

1236

In [88]:
# Merge Series to DataFrame using index
merged_df = recommendation_df.merge(actual_test, left_index=True, right_index=True)
merged_df.rename(columns={'Code_Product': 'Actually_bought'}, inplace=True)

In [89]:
merged_df.head()

,Top_1,Top_2,Top_3,Actually_bought
813a4cf2,5000.5,10031.0,5005.0,[49291.5]
af5f64b4,49292.0,49291.5,5000.5,[49291.5]
b3ce7f25,10031.0,5005.0,10006.0,[45000.5]
f84a035b,5000.5,10031.0,5005.0,[40019.0]
7270dcae,49292.0,49291.5,5000.5,[30002.0]


In [90]:
merged_df['num_items'] = merged_df['Actually_bought'].apply(lambda x: len(x))

In [91]:
merged_df.head()

,Top_1,Top_2,Top_3,Actually_bought,num_items
813a4cf2,5000.5,10031.0,5005.0,[49291.5],1
af5f64b4,49292.0,49291.5,5000.5,[49291.5],1
b3ce7f25,10031.0,5005.0,10006.0,[45000.5],1
f84a035b,5000.5,10031.0,5005.0,[40019.0],1
7270dcae,49292.0,49291.5,5000.5,[30002.0],1


In [92]:
# Count how many elements in the list of the 'Actually_bought' column are present 
# in corresponding 'Top_1' or 'Top_2' or 'Top_3'

merged_df['matched_recommended_items'] = merged_df.apply(
    lambda row: sum(val in [row['Top_1'], \
                            row['Top_2'], \
                            row['Top_3']] for val in row['Actually_bought']), axis=1)

In [93]:
merged_df.head()

,Top_1,Top_2,Top_3,Actually_bought,num_items,matched_recommended_items
813a4cf2,5000.5,10031.0,5005.0,[49291.5],1,0
af5f64b4,49292.0,49291.5,5000.5,[49291.5],1,1
b3ce7f25,10031.0,5005.0,10006.0,[45000.5],1,0
f84a035b,5000.5,10031.0,5005.0,[40019.0],1,0
7270dcae,49292.0,49291.5,5000.5,[30002.0],1,0


In [94]:
merged_df.shape

(2278, 6)

In [95]:
len(new_customers)

1042

In [96]:
new_customer_df = merged_df.loc[new_customers]
existing_customer_df = merged_df.drop(index=new_customers)

In [97]:
new_customer_df.head()

,Top_1,Top_2,Top_3,Actually_bought,num_items,matched_recommended_items
af5f64b4,49292.0,49291.5,5000.5,[49291.5],1,1
7270dcae,49292.0,49291.5,5000.5,[30002.0],1,0
47bb82f4,49292.0,49291.5,5000.5,[49292.0],1,1
e6c48c05,49292.0,49291.5,5000.5,[49356.0],1,0
2257a6e9,49292.0,49291.5,5000.5,[25003.0],1,0


In [98]:
new_customer_df.shape

(1042, 6)

In [99]:
existing_customer_df.head()

,Top_1,Top_2,Top_3,Actually_bought,num_items,matched_recommended_items
813a4cf2,5000.5,10031.0,5005.0,[49291.5],1,0
b3ce7f25,10031.0,5005.0,10006.0,[45000.5],1,0
f84a035b,5000.5,10031.0,5005.0,[40019.0],1,0
996e3cf9,10031.0,5005.0,10006.0,"[45000.5, 40075.5]",2,0
ad02846f,5000.5,10031.0,5005.0,[48513.0],1,0


In [100]:
existing_customer_df.shape

(1236, 6)

In [101]:
existing_customer_df['buy_again'] = existing_customer_df.index.map(filtered_dict)

In [102]:
existing_customer_df.head()

,Top_1,Top_2,Top_3,Actually_bought,num_items,matched_recommended_items,buy_again
813a4cf2,5000.5,10031.0,5005.0,[49291.5],1,0,[35076.0]
b3ce7f25,10031.0,5005.0,10006.0,[45000.5],1,0,"[25003.0, 10015.0, 15000.5, 5000.5]"
f84a035b,5000.5,10031.0,5005.0,[40019.0],1,0,[10023.0]
996e3cf9,10031.0,5005.0,10006.0,"[45000.5, 40075.5]",2,0,"[10015.5, 5001.0, 5000.5, 45004.0, 25003.0, 49..."
ad02846f,5000.5,10031.0,5005.0,[48513.0],1,0,"[45004.0, 49427.5, 15008.5, 40026.5, 49556.0, ..."


In [103]:
# Removing the items which are already bought by the customer before as our recommendation 
# engine wont suggest already bought items as per our design.
existing_customer_df['Actually_bought_modified'] = existing_customer_df.apply(
    lambda row: list(set(row['Actually_bought']) - set(row['buy_again'])), 
    axis=1)

In [104]:
existing_customer_df['num_items_modified'] = existing_customer_df['Actually_bought_modified'].apply(
    lambda x: len(x))

In [105]:
existing_customer_df.head()

,Top_1,Top_2,Top_3,Actually_bought,num_items,matched_recommended_items,buy_again,Actually_bought_modified,num_items_modified
813a4cf2,5000.5,10031.0,5005.0,[49291.5],1,0,[35076.0],[49291.5],1
b3ce7f25,10031.0,5005.0,10006.0,[45000.5],1,0,"[25003.0, 10015.0, 15000.5, 5000.5]",[45000.5],1
f84a035b,5000.5,10031.0,5005.0,[40019.0],1,0,[10023.0],[40019.0],1
996e3cf9,10031.0,5005.0,10006.0,"[45000.5, 40075.5]",2,0,"[10015.5, 5001.0, 5000.5, 45004.0, 25003.0, 49...","[45000.5, 40075.5]",2
ad02846f,5000.5,10031.0,5005.0,[48513.0],1,0,"[45004.0, 49427.5, 15008.5, 40026.5, 49556.0, ...",[48513.0],1


In [106]:
# Count how many elements in the list of the 'Actually_bought_modified' column are present 
# in corresponding 'Top_1' or 'Top_2' or 'Top_3'

existing_customer_df['matched_recommended_items_modified'] = existing_customer_df.apply(
    lambda row: sum(val in [row['Top_1'],\
                            row['Top_2'], \
                            row['Top_3']] for val in row['Actually_bought_modified']), axis=1)

In [107]:
recomm_new = new_customer_df['matched_recommended_items'].sum()/new_customer_df['num_items'].sum()
recomm_old = existing_customer_df['matched_recommended_items'].sum()/existing_customer_df['num_items'].sum()
recomm_old_modified = existing_customer_df['matched_recommended_items_modified'].sum()/existing_customer_df['num_items_modified'].sum()

In [108]:
recomm_new = round(recomm_new*100, 2)
recomm_old = round(recomm_old*100, 2)
recomm_old_modified = round(recomm_old_modified*100, 2)

In [109]:
print(f'{recomm_new}% of the recommended products are bought by the new customers') 
print(f'{recomm_old}% of the recommended products are bought by the existing customers') 
print(f'{recomm_old_modified}% of the recommended products are bought by the existing customers excluding already bought items') 

29.34% of the recommended products are bought by the new customers
4.57% of the recommended products are bought by the existing customers
5.87% of the recommended products are bought by the existing customers excluding already bought items


In [110]:
existing_customer_df[['Top_1', 'Top_2', 'Top_3']].head()

,Top_1,Top_2,Top_3
813a4cf2,5000.5,10031.0,5005.0
b3ce7f25,10031.0,5005.0,10006.0
f84a035b,5000.5,10031.0,5005.0
996e3cf9,10031.0,5005.0,10006.0
ad02846f,5000.5,10031.0,5005.0


In [111]:
new_customer_df[['Top_1', 'Top_2', 'Top_3']].head()

,Top_1,Top_2,Top_3
af5f64b4,49292.0,49291.5,5000.5
7270dcae,49292.0,49291.5,5000.5
47bb82f4,49292.0,49291.5,5000.5
e6c48c05,49292.0,49291.5,5000.5
2257a6e9,49292.0,49291.5,5000.5
